In [1]:
from tensorboard.plugins.hparams import api as hp

In [2]:
import os

import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

from retrieval_recommender import Retriever
from random import sample, choice

from datetime import datetime
import shutil

In [3]:
base_loc = r'D:\dev work\recommender systems\ATRAD_CARS'

train_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/train").cache() #data\ratings_train
test_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test").cache()
portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_tfds").cache()

In [4]:
# model = Recommender(
#     use_timestamp = True,
#     portfolios = portfolios
#     )

In [5]:
# model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.3))

train_ds = train_ds.shuffle(1000).batch(128) #.cache()
test_ds = test_ds.batch(128)

# Tuner

In [6]:
from tensorboard.plugins.hparams import api as hp

In [7]:
log_dir = os.path.join(base_loc ,"logs/retriever_opt")

In [8]:
HP_ITEM_ID_EMBEDDING_DIMS = hp.HParam('item_id_embedding_dims', hp.Discrete([8, 10, 12]))
HP_ITEM_GICS_EMBEDDING_DIMS = hp.HParam('item_gics_embedding_dims', hp.Discrete([8, 10, 12]))
HP_ITEM_NAME_EMBEDDING_DIMS = hp.HParam('item_name_embedding_dims')
HP_USER_ID_EMBEDDING_DIMS = hp.HParam('user_id_embedding_dims', hp.Discrete([16,15]))
HP_USER_TS_EMBEDDING_DIMS = hp.HParam('user_ts_embedding_dims')
HP_EPOCHS = hp.HParam('EPOCHS', hp.Discrete([5,10,20]))

HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

hp_doc_path = os.path.join(log_dir, 'hparam_tuning_{}'.format(datetime.now().strftime("%Y_%m_%d")))

with tf.summary.create_file_writer(hp_doc_path).as_default():
  hp.hparams_config(
    hparams=[
      HP_OPTIMIZER,
      HP_ITEM_ID_EMBEDDING_DIMS,
      HP_ITEM_GICS_EMBEDDING_DIMS,
      HP_ITEM_NAME_EMBEDDING_DIMS,
      HP_USER_ID_EMBEDDING_DIMS,
      HP_USER_TS_EMBEDDING_DIMS,
      HP_EPOCHS
      ],
    metrics=[
        hp.Metric('top_10_categorical_accuracy',
        display_name='top_10_categorical_accuracy')
        ],
  )

In [9]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    top_10_categorical_accuracy = train_test_model(hparams)
    tf.summary.scalar('top_10_categorical_accuracy', top_10_categorical_accuracy, step=1)

In [10]:
def train_test_model(hparams):

  model = Retriever(
    use_timestamp = True,
    portfolios = portfolios,
    item_id_embedding_dim = hparams[HP_ITEM_ID_EMBEDDING_DIMS],
    gics_embedding_dim = hparams[HP_ITEM_GICS_EMBEDDING_DIMS],
    # name_embedding_dim = hparams[HP_ITEM_NAME_EMBEDDING_DIMS],
    user_id_embedding_dim = hparams[HP_USER_ID_EMBEDDING_DIMS],
    # ts_embedding_dim = hparams[HP_USER_TS_EMBEDDING_DIMS]
    )

  

  model.compile(
      optimizer=hparams[HP_OPTIMIZER]
  )

  # model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes

  model.fit(
    train_ds, 
    epochs=hparams[HP_EPOCHS], 
    verbose = 1,
    callbacks=[
        # tf.keras.callbacks.TensorBoard(hp_doc_path),  # log metrics
        # hp.KerasCallback(hp_doc_path, hparams),  # log hparams
    ],
    )
  
  res = model.evaluate(test_ds, return_dict= True)
  # print(res)
  return res['factorized_top_k/top_10_categorical_accuracy']

In [11]:
# deleting previous hp tuning records
shutil.rmtree(hp_doc_path)

# 32 minus is there to keep the embedding size same of users and items.
session_num = 0
combinations = 5
while session_num < combinations:
    
    item_id_embedding_dims = choice(HP_ITEM_ID_EMBEDDING_DIMS.domain.values)
    item_gics_embedding_dims = choice(HP_ITEM_GICS_EMBEDDING_DIMS.domain.values)

    user_id_embedding_dims = choice(HP_USER_ID_EMBEDDING_DIMS.domain.values)

    hparams = {
            HP_OPTIMIZER: choice(HP_OPTIMIZER.domain.values),
            HP_ITEM_ID_EMBEDDING_DIMS: item_id_embedding_dims,
            HP_ITEM_GICS_EMBEDDING_DIMS: item_gics_embedding_dims,
            HP_ITEM_NAME_EMBEDDING_DIMS: (32 - (item_id_embedding_dims + item_gics_embedding_dims)),
            HP_USER_ID_EMBEDDING_DIMS: user_id_embedding_dims,
            HP_USER_TS_EMBEDDING_DIMS: (32 - (user_id_embedding_dims + 1)),
            HP_EPOCHS : choice(HP_EPOCHS.domain.values)
        }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run(os.path.join(hp_doc_path, run_name), hparams)
    session_num += 1

--- Starting trial: run-0
{'optimizer': 'sgd', 'item_id_embedding_dims': 8, 'item_gics_embedding_dims': 8, 'item_name_embedding_dims': 16, 'user_id_embedding_dims': 16, 'user_ts_embedding_dims': 15, 'EPOCHS': 5}
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Epoch 1/5
25/25 [==============================] - 3s 56ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0123 - loss: 572.8158 - regularization_loss: 0.0000e+00 - total_loss: 572.8158
Epoch 2/5
25/25 [==============================] - 1s 54ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0218 - loss: 572.1077 - regularization_loss: 0.0000e+00 - total_loss: 572.1077
Epoch 3/5
25/25 [==============================] - 1s 53ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0188 - loss: 571.7426 - regularization_loss: 0.0000e+00 - total_loss: 571.7426
Epoch 4/5
25/25 [==============================] - 1s 53ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0179 - loss: 571.7662 - regularization_loss: 0.0000e+00 - total_loss: 571.7662
Epoch 5/5
7/7 [==============================] - 1s 55ms/step - factorized_top_k/top_10_categorical_accuracy: 0.0208 - loss: 465.0680 - regularization_loss: 0.0000e+00 - total_loss: 465.0680
--- Starting trial: run-1
{'optimizer

In [12]:
# #\hparam_tuning
# %tensorboard --logdir logs\fit